# 최종 크롤링

In [2]:
import selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
import re

In [3]:
selenium.__version__

'4.12.0'

In [16]:
#지영의 함수 - 최종

from selenium import webdriver
from selenium.webdriver.common.by import By
import re

#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#driver = webdriver.Chrome(options=chrome_options)

def cnbc_crawling(start_date, end_date):

    global driver
    
    start_values_iter = iter(range(0, 320, 10))
    current_matching_links = []
    retry_count = 0
    terminate_flag = False

    while True:
        if terminate_flag:
            break

        try:
            current_start = next(start_values_iter)
        except StopIteration:
            break

        print(f"page: {current_start // 10 + 1}")

        while True:
            matching_links_count = 0
            date_range_str = f"cd_min:{start_date},cd_max:{end_date}"
            google_search_url = f'https://www.google.com/search?q=cnbc&sca_esv=568517199&tbs=cdr:1,{date_range_str}&tbm=nws&sxsrf=AM9HkKmYdZ0_zRMPireMpaE6VsIgqW5jBg:1695740222154&ei=PvESZZmECbfh2roPnbOa-A0&start={current_start}&sa=N&ved=2ahUKEwiZm8rMxMiBAxW3sFYBHZ2ZBt84ygIQ8tMDegQIAxAW&biw=1137&bih=790&dpr=2'
            driver.get(google_search_url)

            links_with_class = driver.find_elements(By.CSS_SELECTOR, 'a.WlydOe[jsname="YKoRaf"]')
            for link in links_with_class:
                href = link.get_attribute('href')
                if href and re.match(r'https://www\.cnbc\.com/\d{4}/\d{2}/\d{2}/[a-z0-9-]+\.html', href):
                    current_matching_links.append(href)
                    print(href)
                    matching_links_count += 1

            if matching_links_count > 0:
                retry_count = 0
                break
            else:
                driver.quit()
                driver = webdriver.Chrome(options=chrome_options)
                retry_count += 1

            #재시도 횟수가 2번 이상이면 - terminate시키고 다음 주로 넘어감. (그 페이지가 비었다는 뜻이므로). 재시도란, 결과가 나오지 않아 드라이버를 껐다 켜는 행위.
            if retry_count >= 2:
                terminate_flag = True
                break
    
    return current_matching_links

#함수 호출
#cnbc_crawling("08/01/2023", "08/31/2023")

###
#driver가 이중으로 열리는 문제 해결

In [17]:
#저장코드 추가

from datetime import datetime, timedelta
import pickle  # make sure to import pickle

base_path = "C:\\Users\\Administrator\\Bitamin_11\\dacon_nh\\dacon_venv\\dataset\\"

# Initialize variables
start_date = datetime.strptime("10/1/2022", "%m/%d/%Y")
end_date = datetime.strptime("9/29/2023", "%m/%d/%Y")
all_matching_links = []
counter = 0  # Initialize counter variable
empty_count = 0

# Driver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

# Generate date ranges
while start_date <= end_date:
    next_end_date = start_date + timedelta(days=6)
    if next_end_date > end_date:
        next_end_date = end_date
    
    # Convert to string format
    current_start_date, current_end_date = (start_date.strftime("%m/%d/%Y"), next_end_date.strftime("%m/%d/%Y"))
    current_matching_links = cnbc_crawling(current_start_date, current_end_date)
    all_matching_links += current_matching_links

    if not current_matching_links:  # Check if the list is empty
        empty_count += 1  # Increment the count
        if empty_count >= 2:  # Check if it happened consecutively twice
            print(f"첫 페이지에서 '로봇이 아닙니다'가 등장하여 코드를 종료합니다 (start_date: {current_start_date - timedelta(days=7)}, end_date: {current_end_date - timedelta(days=7)})")
            break
    else:
        empty_count = 0  # Reset the count if the list is not empty


    # 결과 처리 또는 저장
    print(f'크롤링 결과 (시작 날짜: {current_start_date}, 종료 날짜: {current_end_date}):')
    print(f'누적 크롤링된 기사 수: {len(all_matching_links)}')
    print(f'현재 크롤링된 기사 수: {len(current_matching_links)}')
    
    start_date += timedelta(days=7)
    

    ###저장경로를 변수로 지정해줌
    counter += 1  # Increment counter
    if counter == 4:  # Check if 4 weeks have passed
        file_name = f"all_matching_links_{next_end_date.strftime('%Y_%m_%d')}_retry.pkl"
        full_path = os.path.join(base_path, file_name)
        with open(full_path, 'wb') as f:
            pickle.dump(all_matching_links, f)
        counter = 0  # Reset counter

    # Save the remaining data if next_end_date reaches end_date
if next_end_date == end_date:
    file_name = f"all_matching_links_{next_end_date.strftime('%Y_%m_%d')}_retry.pkl"
    full_path = os.path.join(base_path, file_name)
    with open(full_path, 'wb') as f:
            pickle.dump(all_matching_links, f)


###네트워크 끊겨서 오류난 것


page: 1
https://www.cnbc.com/2022/10/06/adidas-says-its-relationship-with-kanye-west-is-under-review.html
https://www.cnbc.com/2022/10/04/mastercard-deepens-crypto-push-with-tool-for-preventing-fraud.html
https://www.cnbc.com/2022/10/07/jeremy-siegel-the-biggest-threat-today-is-recession-not-inflation.html
https://www.cnbc.com/2022/10/02/tim-cook-says-there-are-4-traits-he-looks-for-in-apple-employees.html
https://www.cnbc.com/2022/10/04/cnbcs-fa-100-recognizes-advisors-who-help-people-make-smart-money-moves.html
https://www.cnbc.com/2022/10/03/pro-russian-groups-are-raising-cryptocurrency-to-prop-up-military-operations.html
https://www.cnbc.com/2022/10/06/mercedes-benz-ceo-says-ev-transition-will-be-helped-by-luxury-drivers.html
https://www.cnbc.com/2022/10/04/nbcuniversal-ceo-jeff-shell-says-movie-business-is-better-with-new-release-model.html
https://www.cnbc.com/2022/10/04/barry-sternlicht-says-unbelievable-calamities-ahead-if-the-fed-keeps-hiking-.html
https://www.cnbc.com/2022/10

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: headless chrome=117.0.5938.132)
Stacktrace:
	GetHandleVerifier [0x00007FF78B227892+54818]
	(No symbol) [0x00007FF78B196AC2]
	(No symbol) [0x00007FF78B04DA3B]
	(No symbol) [0x00007FF78B049853]
	(No symbol) [0x00007FF78B03CB9C]
	(No symbol) [0x00007FF78B03DDF7]
	(No symbol) [0x00007FF78B03CF95]
	(No symbol) [0x00007FF78B03C00E]
	(No symbol) [0x00007FF78B03BFB3]
	(No symbol) [0x00007FF78B03A82E]
	(No symbol) [0x00007FF78B03B123]
	(No symbol) [0x00007FF78B04FE4F]
	(No symbol) [0x00007FF78B0C7917]
	(No symbol) [0x00007FF78B0AEAAA]
	(No symbol) [0x00007FF78B0C75A2]
	(No symbol) [0x00007FF78B0AE883]
	(No symbol) [0x00007FF78B083691]
	(No symbol) [0x00007FF78B0848D4]
	GetHandleVerifier [0x00007FF78B58B992+3610402]
	GetHandleVerifier [0x00007FF78B5E1860+3962352]
	GetHandleVerifier [0x00007FF78B5D9D4F+3930847]
	GetHandleVerifier [0x00007FF78B2C3646+693206]
	(No symbol) [0x00007FF78B1A1628]
	(No symbol) [0

### 저장 코드(이상하니까 안 봐도 됨)

In [5]:
import pandas as pd
import pickle

pkl_all = pd.read_pickle("C:\\Users\\Administrator\\Bitamin_11\\dacon_nh\\dacon_venv\\dataset\\all_matching_links_final.pkl")

In [6]:
print(len(pkl_all))
print(len(set(pkl_all)))

5634
5634


In [3]:
pkl_all = set(pkl_all)
pkl_all

{'https://www.cnbc.com/2023/01/06/russia-ukraine-live-updates.html',
 'https://www.cnbc.com/2023/07/21/espn-had-talks-with-nba-nfl-in-search-for-strategic-partner.html',
 'https://www.cnbc.com/2023/08/31/biden-says-he-will-visit-florida-this-weekend-to-survey-idalia-damage.html',
 'https://www.cnbc.com/2023/05/02/ford-cutting-price-of-electric-mustang-mach-e-by-thousands-of-dollars.html',
 'https://www.cnbc.com/2023/04/20/eu-lawmakers-approve-worlds-first-comprehensive-crypto-regulation.html',
 'https://www.cnbc.com/2023/05/02/one-pilot-planes-are-a-possibility-as-aviation-embraces-ai-emirates-president-says.html',
 'https://www.cnbc.com/2023/06/23/ai-has-a-discrimination-problem-in-banking-that-can-be-devastating.html',
 'https://www.cnbc.com/2023/05/18/disney-worlds-star-wars-galactic-starcruiser-to-shutter.html',
 'https://www.cnbc.com/2023/08/24/south-koreas-naver-launches-generative-ai-services-to-compete-with-chatgpt.html',
 'https://www.cnbc.com/2023/06/16/converting-gas-powered

In [4]:
with open(f'C:\\Users\\Administrator\\Bitamin_11\\dacon_nh\\dacon_venv\\dataset\\all_matching_links_final.pkl', 'wb') as f:
        pickle.dump(pkl_all, f)

In [4]:
import pickle
import pandas as pd

df = pd.read_pickle("C:\\Users\\Administrator\\Bitamin_11\\dacon_nh\\dacon_venv\\dataset\\news_info_ckpt\\100번째_체크포인트.pkl")

0

In [6]:
df[df['url']=='https://www.cnbc.com/2023/03/20/ipcc-report-on-climate-un-scientists-call-for-course-correction.html']

,title,date,category,key_points,text,url
47,World’s top climate scientists issue ‘survival...,2023-03-20,Sustainable Future,The latest report from the U.N.'s Intergovernm...,A local reacts to watching a wildfire advancin...,https://www.cnbc.com/2023/03/20/ipcc-report-on...
